In [1]:
import copernicusmarine
import xarray as xr
import fsspec
import pandas as pd
from seapopym.standard.units import StandardUnitsLabels

/Users/ash/Documents/Workspaces/PhD/Seapopym-optimisation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load DATA


### Observations


In [2]:
papa_obs = xr.open_dataset("./products/Papa_obs.zarr", engine="zarr")
papa_obs

<xarray.Dataset> Size: 2kB
Dimensions:    (time: 69, latitude: 1, longitude: 1, layer: 1)
Coordinates:
  * latitude   (latitude) int64 8B 50
  * layer      (layer) int64 8B 0
  * longitude  (longitude) int64 8B -150
  * time       (time) datetime64[ns] 552B 1997-02-21 1997-06-15 ... 2020-08-20
Data variables:
    day        (time, latitude, longitude, layer) float64 552B ...
    night      (time, latitude, longitude, layer) float64 552B ...
Attributes:
    creation date:  2024-10-13T20:03:40.331216
    layer:          epipelagic
    station:        Papa

In [3]:
LATITUDE = papa_obs.latitude.data[0]
LONGITUDE = papa_obs.longitude.data[0]
# The observation data time range is starting too early. We will use the time range of the model data.
# START_TIME = pd.Timestamp(papa_obs.time.data[0]).strftime("%Y-%m-%d")
# END_TIME = pd.Timestamp(papa_obs.time.data[-1]).strftime("%Y-%m-%d")
START_TIME = "1998-01-01"
END_TIME = "2022-12-31"
DELTA_IMPORT = 0.5
DELTA_COMPUTE = 0.1

print(
    f"Latitude: {LATITUDE}\nLongitude: {LONGITUDE}\nStart Time: {START_TIME}\nEnd Time: {END_TIME}\nDelta import: {DELTA_IMPORT}"
)

Latitude: 50
Longitude: -150
Start Time: 1998-01-01
End Time: 2022-12-31
Delta import: 0.5


### CMEMS


In [4]:
FROM_API = True
path_cmems_product_bio = "..."
path_cmems_product_physic = "..."

In [5]:
# copernicusmarine.login()

coordinates = {
    "minimum_latitude": LATITUDE - DELTA_IMPORT,
    "maximum_latitude": LATITUDE + DELTA_IMPORT,
    "minimum_longitude": LONGITUDE - DELTA_IMPORT,
    "maximum_longitude": LONGITUDE + DELTA_IMPORT,
    "start_datetime": START_TIME,
    "end_datetime": END_TIME,
}


if FROM_API:
    bio = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i", **coordinates)
    bio.load()
    physic = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i", **coordinates)
    physic.load()
else:
    bio = xr.load_dataset(path_cmems_product_bio)
    physic = xr.load_dataset(path_cmems_product_physic)

Fetching catalog: 100%|██████████| 3/3 [00:12<00:00,  4.32s/it]
INFO - 2024-12-11T14:42:15Z - Dataset version was not specified, the latest one was selected: "202411"
INFO - 2024-12-11T14:42:15Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-12-11T14:42:17Z - Service was not specified, the default one was selected: "arco-time-series"
INFO - 2024-12-11T14:42:32Z - Dataset version was not specified, the latest one was selected: "202411"
INFO - 2024-12-11T14:42:32Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-12-11T14:42:33Z - Service was not specified, the default one was selected: "arco-time-series"


In [6]:
data = xr.merge([bio, physic])
lat_attrs, lon_attrs = data.latitude.attrs, data.longitude.attrs
data = data.sel(
    time=slice(START_TIME, END_TIME),
).sel(
    latitude=slice(LATITUDE - DELTA_COMPUTE, LATITUDE + DELTA_COMPUTE),
    longitude=slice(LONGITUDE - DELTA_COMPUTE, LONGITUDE + DELTA_COMPUTE),
)
data = data.mean(dim=["latitude", "longitude"], skipna=True)
data = data.expand_dims({"latitude": [LATITUDE], "longitude": [LONGITUDE]})
data.latitude.attrs, data.longitude.attrs = lat_attrs, lon_attrs
data["T"].attrs["units"] = str(StandardUnitsLabels.temperature.units)
data

<xarray.Dataset> Size: 2MB
Dimensions:              (latitude: 1, longitude: 1, time: 9131, depth: 3)
Coordinates:
  * latitude             (latitude) int64 8B 50
  * longitude            (longitude) int64 8B -150
  * time                 (time) datetime64[ns] 73kB 1998-01-01 ... 2022-12-31
  * depth                (depth) int32 12B 1 2 3
Data variables: (12/13)
    mnkc_epi             (latitude, longitude, time) float64 73kB 1.443 ... 2...
    mnkc_hmlmeso         (latitude, longitude, time) float64 73kB 1.999 ... 2...
    mnkc_lmeso           (latitude, longitude, time) float64 73kB 2.859 ... 3...
    mnkc_mlmeso          (latitude, longitude, time) float64 73kB 1.885 ... 2...
    mnkc_mumeso          (latitude, longitude, time) float64 73kB 2.194 ... 3...
    mnkc_umeso           (latitude, longitude, time) float64 73kB 1.39 ... 2.599
    ...                   ...
    zeu                  (latitude, longitude, time) float64 73kB 58.32 ... 5...
    zooc                 (latitude, longitude, time) float64 73kB 1.969 ... 2...
    T                    (latitude, longitude, time, depth) float64 219kB 5.6...
    U                    (latitude, longitude, time, depth) float64 219kB 0.0...
    V                    (latitude, longitude, time, depth) float64 219kB 0.0...
    pelagic_layer_depth  (latitude, longitude, time, depth) float64 219kB 87....

## Gathering Data & Save


In [7]:
data.to_zarr("./products/Papa_cmems.zarr", mode="w")